In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.naive_bayes import MultinomialNB

import nltk # it is the one of most comman libraries for Natural Language Process
import re # Regular Expression library

In [2]:
data = pd.read_csv("../input/twitter-user-gender-classification/gender-classifier-DFE-791531.csv", encoding="latin1")

# i will use only description column 
data = pd.concat([data.gender, data.description], axis=1)
data.dropna(axis = 0, inplace = True) # we dropped the null rows

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16224 entries, 0 to 20049
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   gender       16224 non-null  object
 1   description  16224 non-null  object
dtypes: object(2)
memory usage: 380.2+ KB


In [4]:
import nltk # natural language took kit
nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
# genders have two options (male/female). so i changed male to 0 and female to 1.
data.gender = [1 if gender == "female" else 0 for gender in data.gender]

In [6]:
import nltk
description_list = [] # we created a list so we after these steps, we will append into this list
for description in data.description:
    description = re.sub("[^a-zA-Z]", " ", description)
    # sub method finds the given pattern ([^a-zA-Z] means, NOT letter like ":") and changes them with " " (space)
    description = description.lower()
    # we need to have all letters lowercase (because A is not equall to a)
    description = nltk.word_tokenize(description)
    # we make a word list from our text
    lemma = nltk.WordNetLemmatizer()
    description = [lemma.lemmatize(word) for word in description]
    # we found the roots of each words with lemma
    description = " ".join(description)
    # after all these steps,we joined the words together and remake our text.
    description_list.append(description)
    # and append these texts into the list we created.

In [7]:
# now, we will implement "bag of words" method
from sklearn.feature_extraction.text import CountVectorizer

max_features = 5000 # we will take top 5000 feature 

cv = CountVectorizer(max_features=max_features, stop_words = "english")
# in this method, we remove the stopwords (irrelevant words) in English language. (like "of", "and", "the" etc.)

sparce_matrix = cv.fit_transform(description_list).toarray()

print("top used {} words: {}".format(max_features, cv.get_feature_names()))

top used 5000 words: ['aa', 'aaron', 'abc', 'aberdeen', 'ability', 'able', 'absolute', 'absolutely', 'absurdity', 'abuse', 'ac', 'academia', 'academic', 'academy', 'acc', 'accept', 'accepted', 'accepting', 'access', 'accessible', 'accessory', 'accident', 'accommodation', 'accomplish', 'account', 'accountant', 'accounting', 'ace', 'achieve', 'act', 'acting', 'action', 'active', 'activist', 'activity', 'actor', 'actress', 'actual', 'actually', 'ad', 'adalah', 'adam', 'add', 'added', 'addict', 'addicted', 'addiction', 'addition', 'address', 'admin', 'administrator', 'admirer', 'adopted', 'adoption', 'adorable', 'adore', 'adult', 'advance', 'advanced', 'adventure', 'adventurer', 'advertising', 'advice', 'advise', 'adviser', 'advisor', 'advocacy', 'advocate', 'advocating', 'aerial', 'aerospace', 'aesthetic', 'af', 'afc', 'affair', 'affiliate', 'affiliated', 'affiliation', 'affordable', 'aficionado', 'afraid', 'africa', 'african', 'afrikaner', 'afro', 'afternoon', 'ag', 'age', 'agency', 'age

These are the most comman words in description list.

In [8]:
y = data.iloc[:, 0].values
x = sparce_matrix

In [9]:
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [10]:
# train test split
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, random_state = 42)

In [11]:
# Random Forest Implementation
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(x_train, y_train)
# prediction
y_pred = rf.predict(x_test)

In [12]:
# Random Forest 
accuracy = 100.0 * accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)

Accuracy:  67.46765249537893


In [13]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(x_train,y_train)
print("accuracy: ",nb.score(x_test,y_test))

accuracy:  0.5163277880468269


# MultinomialNB

In [14]:
nb = MultinomialNB()
nb.fit(x_train, y_train)
# prediction
y_pred = nb.predict(x_test)
accuracy = 100.0 * accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)

Accuracy:  68.20702402957485


# LogisticRegression

In [15]:
lr = LogisticRegression(max_iter = 2000)
lr.fit(x_train,y_train)
# prediction
y_pred = lr.predict(x_test)
accuracy = 100.0 * accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)

Accuracy:  69.56253850893407


# Top accuracy

# XGboost

In [16]:
from xgboost import XGBClassifier
xgb = XGBClassifier(random_state =1)
cv = cross_val_score(xgb,x_train,y_train,cv=5)
print(cv)
print(cv.mean())

[0.67785005 0.67773973 0.67534247 0.68047945 0.68424658]
0.6791316541060718


 # Ensemble Modeling

In [17]:
voting_classfication = VotingClassifier(estimators = [('rf', rf),('lr',lr), ('nb', nb)], voting="hard", n_jobs=-1)
voting_classfication.fit(x_train, y_train)
print("Accuracy: ", voting_classfication.score(x_test,y_test))

Accuracy:  0.6931608133086876


# I gained the top high accuracy with LogisticRegression algorithm for this problem
I hope you enjoy reading my kernel

Please share any suggestions for the further improvement
